<a href="https://colab.research.google.com/github/WillMartin7/edgetpu-ggcnn/blob/main/convert_ggcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Install libraries**
Tensorflow nightly is needed, but last I checked the most up to date version causes an error. Version 2.6.0.dev20210327 seems to work fine.

In [ ]:
!pip install  tf-nightly==2.6.0.dev20210327 keras

     |████████████████████████████████| 454.3MB 37kB/s 
     |████████████████████████████████| 4.0MB 52.8MB/s 
     |████████████████████████████████| 4.0MB 38.1MB/s 
     |████████████████████████████████| 471kB 49.6MB/s 
     |████████████████████████████████| 6.1MB 29.1MB/s 
     |████████████████████████████████| 1.2MB 42.4MB/s 
     |████████████████████████████████| 3.9MB 34.8MB/s 
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.34.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
  Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
  Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
  Found existing installation: gast 0.3.3
    Uninstal

# **Add model and dataset to accessible files**

Model file (e.g. epoch_50_model.hdf5) and dataset file (e.g. dataset_210516_1505.hdf5) need to be placed in google drive folder (e.g. ggcnn_files/)

In [ ]:
# mount google drive to google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Convert to TFLite model**

In [ ]:
import tensorflow as tf
from tensorflow import keras

# load dataset
dataset_fn = '/content/drive/MyDrive/ggcnn_tf/dataset_210516_1505.hdf5'
f = h5py.File(dataset_fn, 'r')

# rgb_imgs = np.array(f['test/rgb']) # remove if below line works
rep_imgs = np.array(f['test/rgb'], dtype=np.float32)

#representative dataset
def representative_dataset():
  for data in tf.data.Dataset.from_tensor_slices((rep_imgs)).batch(1).take(150):
      yield [data]

# load model
model_checkpoint_fn = '/content/drive/MyDrive/ggcnn_tf/epoch_50_model.hdf5'
model = keras.models.load_model(model_checkpoint_fn)
model.summary()

# convert model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
converter.experimental_new_converter = False
tflite_quant_model = converter.convert()

2.6.0-dev20210327
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 100, 100, 32) 7808        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 50, 50, 16)   12816       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 25, 25, 8)    1160        conv2d_1[0][0]                   
____________________________________________________________________________

# **Convert to Edge TPU model**

In [ ]:
# install edge tpu compiler: https://coral.ai/docs/edgetpu/compiler/ 
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update
!sudo apt-get install edgetpu-compiler

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1974  100  1974    0     0  21933      0 --:--:-- --:--:-- --:--:-- 21933
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [6,722 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:9 htt

In [ ]:
# write tflite model to file
open('ggcnn_model.tflite', 'wb').write(tflite_quant_model)

# convert and save model as ggcnn_model_edgetpu.tflite
!edgetpu_compiler -s ggcnn_model.tflite

Edge TPU Compiler version 15.0.340273435

Model compiled successfully in 1821 ms.

Input model: model.tflite
Input size: 76.01KiB
Output model: model_edgetpu.tflite
Output size: 988.86KiB
On-chip memory used for caching model parameters: 195.25KiB
On-chip memory remaining for caching model parameters: 7.16MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 13
Operation log: model_edgetpu.log

Operator                       Count      Status

RELU                           3          Mapped to Edge TPU
TRANSPOSE_CONV                 3          Mapped to Edge TPU
CONV_2D                        7          Mapped to Edge TPU
